Click the button below if you want to see the code behind the widgets. If you do click  and want to go back to the 'no-code' view, scroll down until you find the 'hide code' button.

In [1]:
"""
MIT License

Copyright (c) 2020 Sylvain Barde - University of Kent

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams
from IPython.display import display, HTML

javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Click to show code", True: "Click to hide code"}

def toggle_code(state):
    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """

    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)

    display(HTML(output))
    
def button_action(value):
    """
    Calls the toggle_code function and updates the button description.
    """
    state = value.new

    toggle_code(state)

    value.owner.description = button_descriptions[state]


def total_revenue_widget(Qmax_init = 15, Pmax_init = 30, Qval_init = 7.5, a_d_init = 2, b_d_init = 25):

    # Declare widgets for interactive input
    Qmax_slider = widgets.IntSlider(min= 5,
                                 max=50,
                                 step=1,
                                 description=r'Maximum $Q$:',
                                 value = Qmax_init,
                                 continuous_update =False)
    Pmax_slider = widgets.IntSlider(min=5,
                                 max=50,
                                 step=1,
                                 description=r'Maximum $P$:',
                                 value = Pmax_init,
                                 continuous_update =False)
    Qval_slider = widgets.FloatSlider(min=0,
                                 max=50,
                                 description='Pick a Quantity:',
                                 value = Qval_init,
                                 continuous_update =False)
    a_d_slider = widgets.FloatSlider(min=0,
                                 max=10,
                                 description= r'$a_d$:',
                                 value = a_d_init,
                                 continuous_update =False)
    b_d_slider = widgets.FloatSlider(min=0,
                                 max=50,
                                 description= r'$b_d$:',
                                 value=b_d_init,
                                 continuous_update =False)
    
    # Link widgets as required
    widgets.jslink((Qmax_slider,'value'),(Qval_slider,'max'))

    def total_revenue_plot(Qmax,Pmax, Qval, a_d, b_d):

        # create a quantity vector, calculate demand/Total revenue vectors
        Q = np.arange(0,Qmax,Qmax/500)

        P_d = -a_d*Q + b_d
        TR = -a_d*Q**2 + b_d*Q

        # Calculate prices/revenues for selected value
        Pval_d = -a_d*Qval + b_d
        TRval = -a_d*Qval**2 + b_d*Qval
        if a_d == 0:
            TRmax = 1.25*b_d*Qmax
        else:
            TRmax = 1.25*(b_d**2)/(4*a_d)

        # Create figure
        mrkrSize = 2*rcParams['lines.markersize'] ** 2
        fig, ax = plt.subplots(nrows=1, ncols=2,figsize=(20,10))

        # Plot demand
        ax[0].plot(Q, P_d,'b', linewidth=2, alpha=0.6,
                    label=r'Inverse Demand $\quad P_d = -a_d Q + b_d$')

        # Add markers for the price/quantity points, with dotted lines
        ax[0].fill([0,Qval,Qval,0],[0,0,Pval_d,Pval_d],'b',alpha = 0.2,
                        label = r'Total revenue $P_d \times Q$')
        ax[0].scatter(Qval, Pval_d, s=mrkrSize, c='k', alpha=0.6,
                    label='Selection')
        ax[0].plot([0,Qval],[Pval_d,Pval_d],'k--',linewidth=1)
        ax[0].plot([Qval,Qval],[0,Pval_d],'k--',linewidth=1)
        ax[0].annotate(r'$Q={:.2f}$'.format(Qval),[Qval,0], 
                    xytext = [Qval+0.15,0.25], xycoords ='data', fontsize = 25, 
                    clip_on = True)
        ax[0].annotate(r'$P_d={:.2f}$'.format(Pval_d),[0,Pval_d], 
                    xytext = [0.15,Pval_d+0.25], xycoords ='data', fontsize = 25, 
                    clip_on = True)

        # Add legend and format axes to look nice
        ax[0].legend(loc='upper center', frameon=False,prop={'size':20})
        ax[0].autoscale(enable=True, axis='both', tight=True)
        ax[0].set_ylim(top = Pmax, bottom = 0)
        ax[0].set_xlim(right = Qmax, left = 0)
        ax[0].spines['top'].set_visible(False)
        ax[0].spines['right'].set_visible(False)
        ax[0].set_xlabel(r'$Q$', fontdict = {'fontsize': 25},position=(1, 0))
        ax[0].set_ylabel(r'$P$', fontdict = {'fontsize': 25},position=(0, 1), rotation=0)
        ax[0].plot(1, 0, ">k", transform=ax[0].get_yaxis_transform(), clip_on=False)
        ax[0].plot(0, 1, "^k", transform=ax[0].get_xaxis_transform(), clip_on=False)
        ax[0].tick_params(labelsize=20)

        # Plot Total revenue
        ax[1].plot(Q, TR,'b', linewidth=2, alpha=0.6,
                    label=r'Total revenue $\quad TR = -a_d Q^2 + b_d Q$')


        # Add markers for the price/quantity points, with dotted lines
        ax[1].scatter(Qval, TRval, s=mrkrSize, c='k', alpha=0.6,
                    label='Selection')
        ax[1].plot([0,Qval],[TRval,TRval],'k--',linewidth=1)
        ax[1].plot([Qval,Qval],[0,TRval],'k--',linewidth=1)
        ax[1].annotate(r'$Q={:.2f}$'.format(Qval),[Qval,0], xytext = [Qval+0.15,0.25],
                    xycoords ='data', fontsize = 25, clip_on = True)
        ax[1].annotate(r'$TR={:.2f}$'.format(TRval),[0,TRval], xytext = [0.15,TRval+0.25],
                    xycoords ='data', fontsize = 25, clip_on = True)

        # Add legend and format axes to look nice
        ax[1].legend(loc='upper center', frameon=False,prop={'size':20})
        ax[1].autoscale(enable=True, axis='both', tight=True)
        ax[1].set_ylim(top = TRmax, bottom = 0)
        ax[1].set_xlim(right = Qmax, left = 0)
        ax[1].spines['top'].set_visible(False)
        ax[1].spines['right'].set_visible(False)
        ax[1].set_xlabel(r'$Q$', fontdict = {'fontsize': 25},position=(1, 0))
        ax[1].set_ylabel(r'$TR$', fontdict = {'fontsize': 25},position=(0, 1), rotation=0)
        ax[1].plot(1, 0, ">k", transform=ax[1].get_yaxis_transform(), clip_on=False)
        ax[1].plot(0, 1, "^k", transform=ax[1].get_xaxis_transform(), clip_on=False)
        ax[1].tick_params(labelsize=20)  

        plt.tight_layout()
    
    out = widgets.interactive_output(total_revenue_plot, {'Qmax': Qmax_slider,
                                                       'Pmax': Pmax_slider,
                                                       'Qval': Qval_slider, 
                                                       'a_d': a_d_slider,
                                                       'b_d': b_d_slider})

    output = widgets.VBox([out,
                  widgets.HBox([Qmax_slider,
                               Pmax_slider,
                               Qval_slider]),
                  widgets.HBox([a_d_slider, 
                                b_d_slider])])
    display(output)    
    

In [2]:
state = False
toggle_code(state)

button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")

display(button)

ToggleButton(value=False, description='Click to show code')

### Total revenue: A simple application of quadratic functions in economics

You might be glad to hear that this week's application will be a lot shorter than the matrix examples we saw last week. The purpose of the notebook today is to use the concept of total revenue (TR) to illustrate a point I made in the videos: how do quadratic relationships arise from initially linear relationships? 

As I explained in the video, there are two ways this can happen, and we will look at both
1. A quadratic equation can arise from the product of two linear equations
2. It can also arise from a ratio of variables, when the ratio is linear in one of the variables.

#### Total revenue as the product of price and quantity

Let's look at the first possibility: a product of linear functions. A good place to start would be to define what total revenue is! The total revenue of a firm is the revenue generated by selling its output on the market. If we assume that all the output is sold at the same price, then total revenue defined simply as:

$$TR = P\times Q$$

Hopefully, nothing complicated here! If you produce 1000 units and sell them at £50 a piece, then your total revenue is £50 000. So far so good. What shape does total revenue take , as a function of quantity produced? Well, that depends on whether or not the price $P$ is constant. I've provided a diagram to illustrate the two possible cases.
- As you will see in your principles of economics class, if we assume **perfect competition**, then firms are price takers. They are to small relative to the size of the market to influence the market price, and so have to take the market price as given. From the point of view of a firm in perfect competition, the price is a constant $p$ and so total revenue is a simple linear function of the quantity produced by the firm: $TR = pQ$. 
    - *Note:* In the market diagram on the left, because price is measured on the vertical axis and quantity is measured on the horizontal axis, the product of the two is the area of the blue rectangle.



In [3]:
total_revenue_widget(Qval_init = 10, a_d_init = 0, b_d_init = 15)

- If, however, we assume the firm is large enough that changes in its level of output $Q$ can influence the market price, then the shape changes and a quadratic relationship can emerge.

To make things simpler, let's assume a monopoly, where a single firm produces for the entire market. The demand curve facing this firm is the demand curve for the entire market, and it is therefore downward sloping. Following what we did in previous weeks we'll assume that the inverse demand function is.

$$P_d = -a_d Q+b_d$$

Clearly, the market price depends negatively on the quantity produced by the firm. If it reduces production, this will tend to increase prices: as the good becomes rarer, consumers with higher willingness to pay will bid the price up. Similarly, if the monopolist wants to sell more output it will have to reduce prices to attract more buyers. Given this, we can replace the inverse demand function in the expression for total revenue, and a quadratic relationship appears.

$$\begin{aligned}
TR & = P_d\times Q \\
TR & = (-a_d Q+b_d) Q \\
TR & = -a_d Q^2+b_d Q \\
\end{aligned}$$


In [4]:
total_revenue_widget()



The diagram above shows the case where the downward sloping demand curve leads to a quadratic total revenue. The quadratic in this case emerges from the product of price $P$, which is a linear function of quantity $Q$, and of quantity itself, which is trivially a linear equation in $Q$...

An interesting aspect of the total revenue function is that it corresponds to one of the special cases we saw in the material, where the quadratic has no intercept. Let's look at one of the equations from the derivation above:

$$TR = (-a_d Q+b_d) Q $$

This is actually the factorised version of the total revenue equation, and it gives us the roots of the quadratic equation. $TR=0$ for the following quantities:

$$Q = 0 \qquad {\rm and} \qquad Q = \frac{b_d}{a_d} $$

If you want you can play around with the quantity slider in the diagram to follow the discussion. It should be quite obvious that when nothing is sold and $Q=0$ then there will be no revenue generated! This is the fundamental reason ***why*** our quadratic function is this 'no-intercept' special case: if you don't sell anything, you're not going to have revenue...

As you increase your production (and the quantity slider) total revenue start increasing, but it will eventually peak and start declining. This is because the price falls as the quantity sold increases and at some point the negative effect of the falling price on total revenue will more than offset the positive effect of selling more goods. Eventually, when it gets to the point where the firm has to give away the output for free to clear the large quantities it has produced, total revenue will be zero. Again, this should make sense: if you give away your output for free, you're not going to have revenue...

#### Average revenue as the ratio of total revenue quantity

As we discussed in the videos, a quadratic function can also emerge when you have a ratio of variables and that ratio depends linearly on the variable in the denominator. In our case, the ratio of interest is the average revenue that the firm generates by selling its goods. Average revenue is simply total revenue divided by quantity.

$$AR = \frac{TR}{Q}$$

Now if we stick to our simple example (see diagrams above) where all the output produced is sold at the same price $P_d$, then average revenue per unit sold is simply the price:

$$AR = \frac{TR}{Q} = \frac{P_d \times Q}{Q} = P_d$$

If we replace the definition of average revenue by $TR/Q$ and price by the inverse demand function, we recover the same quadratic as before:

$$\begin{aligned}
\frac{TR}{Q} & = -a_d Q+b_d \\
TR & = (-a_d Q+b_d) Q \\
TR & = -a_d Q^2+b_d Q \\
\end{aligned}$$

This second example might seem a bit contrived: after all, it seems we've just moved the terms around the same equation. In reality, this actually a very general way of generating a quadratic relationship. The reason everything is simply is because we have assumed that all the output produced is sold at the same price $P_d$. In general, if we try to think about a real-life firm, the same output will often be sold at different prices (cinema tickets, airline seats etc.) When this is the case there is no longer a single price and total revenue is no longer simply $P\times Q$. Average revenue, however, remains equal to total revenue divided by quantity. As long as this average revenue falls linearly with quantity (which it should), then we still get a quadratic equation for total revenue, despite no longer having the simple $TR = P\times Q$ relation.


#### Wrapping up...

So this was a quick visualisation designed to show you how even in a market where you have straight demand curves, you can find variables that are defined with a quadratic relationship, simply from the fact that you are multiplying or dividing linear equations.

We can also draw some more general conclusions relating to the concept of total revenue. In the example diagrams, we saw that  total revenue is a quadratic function of quantity. You may object (and you'd be right) that this relies on the inverse demand function, or the average revenue function being straight lines, which is unlikely in the real world. This is entirely correct, however using a more realistic demand function will not change the general 'look' of the $TR$ curve. It it is the case that price given by the demand curve falls as quantity on the market increases (whether or not it it a straight line), then in general the total revenue curve will be an inverse U shape. This is because by definition total revenue ***must*** be 0 when quantity is 0, and it ***must*** be zero when the market is so saturated with output that the price has fallen to 0. So it ***must*** be that total revenue initially increases, has a maximum, then falls down again. It it entirely possible that the *exact* curve is a not quadratic in general cases, but it does mean that a quadratic will usually be a good approximation.

